In [4]:
# notebook setup
import datetime as dt
from io import StringIO
import requests
import os
import sys

from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
from pathlib import Path
stdout = sys.stdout
p = Path().resolve().parents[1]
sys.path.append(str(p))
import gscap.gps as gps
sys.stdout = stdout
InteractiveShell.ast_node_interactivity = 'all'
np.random.seed(42)

C:\Users\calvi\AppData\Local\Temp\ipykernel_2672\1051505780.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## The GSCAP GPS scripts 
provide a number of  miscellaneous, clustering, and api wrapper methods. The main format of each call is a Pandas dataframe containing three columns: lat, lon, and ts like shown in the next cell.

In [5]:
r = requests.get('https://raw.githubusercontent.com/UW-Creativ-Lab/gSCAP/master/notebooks/demos/some_data.csv')

if r.ok:
    sio = StringIO(str(r.content,'utf-8'))    
    df = pd.read_csv(sio, parse_dates=['ts'])
    df.head()
    
    sio.close()
    del sio

,lat,lon,ts
0,45.50473,-122.78290,2015-06-01 00:01:04.561
1,45.50473,-122.78290,2015-06-01 00:02:16.037
2,45.50471,-122.78284,2015-06-01 00:03:01.134
3,45.50471,-122.78284,2015-06-01 00:07:51.359
4,45.50469,-122.78291,2015-06-01 00:09:10.460


#### `resample_gps_intervals(records)`
Currently, this method resamples at one-minute intervals taking the mean of each lat, long within the interval. The timestamp returned is the minute for which the sampling occurred.

In [6]:
df = gps.resample_gps_intervals(df)
df.head()

,lat,lon,ts
0,45.50473,-122.78290,2015-06-01 00:01:00
1,45.50473,-122.78290,2015-06-01 00:02:00
2,45.50471,-122.78284,2015-06-01 00:03:00
3,45.50471,-122.78284,2015-06-01 00:07:00
4,45.50469,-122.78291,2015-06-01 00:09:00


#### `impute_stationary_coordinates(records, freq)` 
This method can be used to upsample the number of points in a stationary location. This is used to account for specific data collection methods that don't take samples if the research participant hasn't moved locations. This is beneficial when used with the density based clustering techniques defined below.
* freq - is a given interval as defined in the Pandas timeseries methods. Read more in the [Pandas Docs](https://pandas.pydata.org/pandas-docs/stable/timeseries.html)

In [7]:
df = gps.impute_stationary_coordinates(df, freq='10Min')
df.head()

processing metrics: 100%|███████████████████████████████████████████████████████| 47556/47556 [01:27<00:00, 546.21it/s]


,lat,lon,ts,displacement,time_delta,velocity,binning,date
0,45.50473,-122.78290,2015-06-01 00:01:00,NaN,NaN,NaN,null,2015-06-01
1,45.50473,-122.78290,2015-06-01 00:02:00,0.0,60.0,0.000,stationary,2015-06-01
2,45.50471,-122.78284,2015-06-01 00:03:00,5.2,60.0,0.086,stationary,2015-06-01
3,45.50471,-122.78284,2015-06-01 00:07:00,0.0,240.0,0.000,stationary,2015-06-01
4,45.50469,-122.78291,2015-06-01 00:09:00,5.9,120.0,0.049,stationary,2015-06-01


### Clustering

#### `get_clusters_with_context(records)`
Again, this takes the same arguments as above.  This method partitions the records into three sets - home, work, and everything else. Then, performs separate clustering on each partition. Two variables are returned. The first is the records with an additional row assigning cluster cids, and the second is the clusters themselves.

This function utilizes the unsupervised clustering algorithm DBSCAN and the provided records. Parameters should be a dict and contain both min_samples and eps. See the [scikit-learn docs](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) for more information. Default parameters will be used if none are supplied. However, these parameters were tuned to generalize well on a specific dataset and I recommend doing the same. Some good sources for tuning DBSCAN hyperparameters are:

* x
* y

In [8]:
df, clusters = gps.get_clusters_with_context(df)
df.head()

,lat,lon,ts,displacement,time_delta,velocity,binning,date,cid,distance_from_home
0,45.50473,-122.78290,2015-06-01 00:01:00,NaN,NaN,NaN,null,2015-06-01,xNot,4.515553
0,45.50473,-122.78290,2015-06-01 00:02:00,0.0,60.0,0.000,stationary,2015-06-01,home,4.515553
1,45.50471,-122.78284,2015-06-01 00:03:00,5.2,60.0,0.086,stationary,2015-06-01,home,8.618027
2,45.50471,-122.78284,2015-06-01 00:07:00,0.0,240.0,0.000,stationary,2015-06-01,home,8.618027
3,45.50469,-122.78291,2015-06-01 00:09:00,5.9,120.0,0.049,stationary,2015-06-01,home,8.895594


In [9]:
clusters = clusters.sort_values(by='cid')
clusters.head()

,lat,lon,cid,name,categories
18,45.50477,-122.78291,home,home,home
19,45.51163,-122.68454,work,work,work
0,45.51224,-122.68435,x0,nap,nap
1,45.54832,-122.65123,x1,nap,nap
10,45.46099,-122.70129,x10,nap,nap


### Place lookup

The place methods are currently built to allow calls to either Yelp or Google for contextual lookup. Both are processed via the same call and require either to be given with in the form of a list of `gps.PlaceRequest`. Each request is cached to a backend sqlite file so that the same request doesn't have to be submitted more than once. Results are returned a dictionary containing three keys:
1. `hits` tells you how many of your requests were returned from the cache
2. `misses` tells you how many went through the API endpoint 
3. `request` is a dataframe of the resulting places

An enum is implemented within the gps module that allows easy selection of an API source. The possibilities are: 
1. `gps.ApiSource.YELP` 
2. `gps.ApiSource.GOOGLE`

With either, a value for `rankby` should also be supplied.  For Google, only prominence is currently implemented - `gps.GmapsRankBy.PROMINENCE`. A number of options are available for Yelp:
* `gps.YelpRankBy.BEST_MATCH`
* `gps.YelpRankBy.RATING`
* `gps.YelpRankBy.REVIEW_COUNT`
* `gps.YelpRankBy.DISTANCE`

#### `request_nearby_places(request)`

In [10]:
xone = list(clusters.loc[clusters.cid=='x0'].itertuples())[0]
request = gps.PlaceRequest(
    lat=xone.lat,
    lon=xone.lon, 
    radius=50, 
    source=gps.ApiSource.YELP,
    rankby=gps.YelpRankBy.BEST_MATCH,
)
results = gps.request_nearby_places(request)
results['request']

C:\Users\calvi\OneDrive\Documents\thesi\gSCAP\.gscapConfig


,lat,lon,radius,source,dtRetrieved,name,rank_order,categories,major_categories
0,45.51224,-122.68435,50,Yelp,2022-02-20 18:12:13.693806,Farewell to Orpheus,0.0,landmarks,government_offices


### Gathering metrics

#### `get_cluster_times(records, clusters)`
Calculates daily entry and exit times a research participant makes into their defined clusters.

In [ ]:
entries = gps.get_cluster_times(df)
entries.head()

#### `get_daily_metrics(records, entries)`
Uses the records set and  daily features

In [ ]:
daily_metrics = gps.get_daily_metrics(df)
daily_metrics.head()

### Miscellaneous

`dd_from_zip(zipcode)`. This method allows you to get a latitude and longitude pair by providing a zipcode.

In [ ]:
gps.dd_from_zip(47579)

`zip_from_dd(lat, lon)`. provides the opposite conversion - from lat, lon back to a zipcode.

In [ ]:
gps.zip_from_dd(32.3788, -84.90685)

Another extremely useful method is to get the Pandas timezone from a lat, lon pair or zipcode. This functionality is provided with `tz_from_zip` and `tz_from_dd`.

In [ ]:
gps.tz_from_dd((32.3788, -84.90685))
gps.tz_from_zip(98115)

Often you want to process entire columns of lat, lon pairs from a dataframe at once.

In [ ]:
gps.tz_from_dd(df[['lat', 'lon']])

This also works for a series of zipcodes.

In [ ]:
gps.tz_from_zip([47579, 98115, 2345])

#### `geo_distance(lat1, lon1, lat2, lon2)` 
Computes the geographic distance in meters between two latitude, longitude pairs.

In [ ]:
pair1 = (47.1, 122.1)
pair2 = (47.2, 122.2)

gps.geo_distance(*pair1, *pair2)

#### `geo_pairwise_distances(x, n_jobs)`
Accepts a list of coordinate pairs and computes the pairwise geographic distance between each pair. Set `n_jobs` to -1 to use all available cores.

In [ ]:
x = [
    (47.111, 122), 
    (47.112, 122),
    (47.113, 122)
]

gps.geo_pairwise_distances(x, as_array=True, n_jobs=-1)